<a href="https://colab.research.google.com/github/AllysonAndrey/Microdados_Enem/blob/main/Data_Normalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
caminho_base= f'/content/drive/MyDrive/ENEM/Enem_Lakehouse/'
pasta_bronze = f'{caminho_base}01-bronze'
pasta_silver = f'{caminho_base}02-silver'

# vamos pegar o arquivo da pasta base (dados brutos .csv)
imput_dir = pasta_bronze

# salvar com parquet na pasta silver
output_dir = pasta_silver

os.makedirs(output_dir, exist_ok=True) # garante que a pasta exista

list_files = os.listdir(imput_dir) # lista os arquivos da pasta base

----

In [ ]:
#link do dicionario: https://docs.google.com/spreadsheets/d/1athUzsch-vFc7DKIdMDpOvapkbx4bEfr_Q4ncKcOU8k/edit?usp=sharing
sheet_id = '1athUzsch-vFc7DKIdMDpOvapkbx4bEfr_Q4ncKcOU8k'
sheet_name_l = 'Colunas_Imutaveis'
sheet_name_n = 'Colunas_Imutaveis_Numeros'
url = f' https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_l}'
url_n = f' https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_n}'

In [ ]:
dic_imutaveis_l = pd.read_csv(url) #, header=1)
dic_imutaveis_n = pd.read_csv(url_n) #, header=1)

In [ ]:
#Preenchendo a coluna Nome_Variavel com os dados que existem na mesma coluna e estão acima dos resultados que estão NaN
dic_imutaveis_l['Nome_Variavel'] = dic_imutaveis_l['Nome_Variavel'].ffill()
dic_imutaveis_n['Nome_Variavel'] = dic_imutaveis_n['Nome_Variavel'].ffill()

# Resetando os index
dic_imutaveis_l = dic_imutaveis_l[['Nome_Variavel','Categoria','Descricao_Categoria']].reset_index(drop = True)
dic_imutaveis_n = dic_imutaveis_n[['Nome_Variavel','Categoria','Descricao_Categoria']].reset_index(drop = True)

#Criando uma lista dos valores que existem na coluna Nome_Variavel, para saber quais as colunas estão impostas
imutaveis_l = dic_imutaveis_l['Nome_Variavel'].unique()
imutaveis_n = dic_imutaveis_n['Nome_Variavel'].unique()

lista_dimensoes_l = list( set(imutaveis_l)  )
lista_dimensoes_n = list( set(imutaveis_n)  )

# Organizando a lista de forma abc
lista_dimensoes_n = sorted(lista_dimensoes_n)
lista_dimensoes_l = sorted(lista_dimensoes_l)

In [ ]:
def add_categ_nulo(dicionario, df, lista_dimensoes, col_nome, col_codigo, col_desc):
    #
    #Função universal que:
    #1. Verifica o tipo de dados que existe na coluna no dataframe fato
    #2. Caso seja numero (int ou float), então ele soma +1 e cria uma linha no dataframe do dicionario
    #3. Caso seja letra (object ou string) , então ele insere um X
    #4. E como descrição, ele Coloca NULO em ambas as opções

    #Parâmetros:
    #- df: DataFrame com os dados
    #- dicionario: DataFrame com os códigos (dicionário)
    #- lista_colunas: lista das colunas para processar
    #- col_nome: nome da coluna que contém os nomes das variáveis no dicionário
    #- col_codigo: nome da coluna que contém os códigos no dicionário
    #- col_desc: nome da coluna que contém as descrições no dicionário
    #

    dicionario_atualizado = dicionario.copy()

    # Cria as novas linhas baseado nos valores máximos do DataFrame enem
    novas_linhas = []
    for dimensao in lista_dimensoes:
        if dimensao in df.columns:
            # Verifica o tipo de dado da coluna
            tipo_coluna = df[dimensao].dtype

            # Se for numérico (int, float)
            if pd.api.types.is_numeric_dtype(tipo_coluna):
                # Encontra o valor máximo na coluna do DataFrame enem (ignorando NaN)
                # Converte para numérico para garantir comparação correta
                valores_numericos = pd.to_numeric(df[dimensao], errors='coerce')
                max_valor = valores_numericos.max()

                # Só adiciona se encontrou um valor válido (não NaN)
                if pd.notna(max_valor):
                    # Também verifica se já existe essa combinação no dicionário original
                    max_dicionario = dicionario_atualizado[dicionario_atualizado[col_nome] == dimensao][col_codigo].max()
                    if pd.notna(max_dicionario):
                        max_valor = max(max_valor, max_dicionario)

                    nova_linha = {
                        col_nome: dimensao,
                        col_codigo: int(round(max_valor)) + 1,  # Incrementa +1 do valor máximo encontrado
                        # colocado o round para o tratamento das colunas float
                        col_desc: 'Nulo'
                    }
                    novas_linhas.append(nova_linha)


            # Se for object ou string
            elif tipo_coluna == 'object' or pd.api.types.is_string_dtype(tipo_coluna):
                nova_linha = {
                    col_nome: dimensao,
                    col_codigo: 'X',  # Coloca 'X' como código
                    col_desc: 'Nulo'  # Coloca 'NULO' como descrição
                }
                novas_linhas.append(nova_linha)

    # Adiciona as novas linhas ao DataFrame original
    if novas_linhas:  # Só concatena se houver novas linhas
        df_novas_linhas = pd.DataFrame(novas_linhas)
        dic_atualizado = pd.concat([dicionario_atualizado, df_novas_linhas], ignore_index=True)
    else:
        dic_atualizado = dicionario_atualizado

    # Ordena por Nome_Variavel e depois por Categoria
    dic_atualizado = dic_atualizado.sort_values([col_nome, col_codigo]).reset_index(drop=True)

    return dic_atualizado


In [ ]:
def substituir_valores_ausentes ( dicionario,df, lista_colunas, col_nome, col_codigo, col_desc):

    #
    #Função universal que:
    #1. Converte colunas float para Int64 se necessário
    #2. Substitui NaN e <NA> pelos códigos NULO correspondentes
    #3. Funciona com qualquer tipo de valor ausente

    #Parâmetros:
    #- df: DataFrame com os dados
    #- dicionario: DataFrame com os códigos (dicionário)
    #- lista_colunas: lista das colunas para processar
    #- col_nome: nome da coluna que contém os nomes das variáveis no dicionário
    #- col_codigo: nome da coluna que contém os códigos no dicionário
    #- col_desc: nome da coluna que contém as descrições no dicionário
    #

    df_resultado = df.copy()

    for coluna in lista_colunas:
        if coluna not in df_resultado.columns:
            print(f"Coluna {coluna} não encontrada no DataFrame")
            continue

        # Verificar se a coluna tem valores ausentes
        tem_ausentes = df_resultado[coluna].isna().any() or (df_resultado[coluna] == '<NA>').any()

        if not tem_ausentes:
            print(f"{coluna}: sem valores ausentes")
            continue

        # Encontrar o código NULO para esta coluna específica
        codigo_nulo_rows = dicionario[
            (dicionario[col_nome] == coluna) &
            (dicionario[col_desc] == 'NULO')
        ]

        if codigo_nulo_rows.empty:
            print(f"AVISO: {coluna} não tem código NULO no dicionário")
            continue

        codigo_nulo = codigo_nulo_rows[col_codigo].iloc[0]

        # Verificar o tipo da coluna e converter se necessário
        if df_resultado[coluna].dtype == 'float64':
            # Converter para Int64 (suporta NA)
            df_resultado[coluna] = df_resultado[coluna].astype('Int64')
            print(f"{coluna}: convertido de float64 para Int64")

        # Contar valores ausentes antes
        ausentes_antes = df_resultado[coluna].isna().sum()

        # Substituir todos os tipos de valores ausentes
        df_resultado[coluna] = df_resultado[coluna].fillna(codigo_nulo)

        # Se ainda tem <NA> como string, substituir também
        if df_resultado[coluna].dtype == 'object':
            df_resultado[coluna] = df_resultado[coluna].replace('<NA>', codigo_nulo)

        # Contar valores ausentes depois
        ausentes_depois = df_resultado[coluna].isna().sum()

        print(f"{coluna}: {ausentes_antes} valores ausentes substituídos por código {codigo_nulo}")

        if ausentes_depois > 0:
            print(f"AVISO: {coluna} ainda tem {ausentes_depois} valores ausentes")

    return df_resultado

In [ ]:
arquivo_testew = 'MICRODADOS_ENEM_2021.csv'
file = arquivo_testew
input_path = os.path.join(imput_dir, file)
output_path = os.path.join(output_dir, file.replace(".csv", ".parquet"))

df_enem_base = pd.read_csv(input_path, sep=";", encoding='ISO-8859-1', nrows= 1500000)

colunas_para_retira = ['TX_GABARITO_CH','TX_GABARITO_CN','TX_GABARITO_MT','TX_GABARITO_LC','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT','TX_RESPOSTAS_CN']
df_enem = df_enem_base.drop(columns=colunas_para_retira)

# Combinar as listas vindas da planilha
todas_dimensoes = lista_dimensoes_l + lista_dimensoes_n

# Combinar os dicionários
dic_combinado = pd.concat([dic_imutaveis_l, dic_imutaveis_n], ignore_index=True)

# Processar tudo de uma vez
dic_final = add_categ_nulo(
        dic_combinado, df_enem, todas_dimensoes,
        'Nome_Variavel', 'Categoria', 'Descricao_Categoria'
    )

enem_final = substituir_valores_ausentes(
        dic_final, df_enem, todas_dimensoes,
        'Nome_Variavel', 'Categoria', 'Descricao_Categoria'
    )

lista_local  = ['CO_MUNICIPIO_ESC','NO_MUNICIPIO_ESC','CO_UF_ESC','SG_UF_ESC']
lista_provas = ['CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT']

# Combinar as listas de colunas que possivelmente terão dados em branco ainda e não serão dados calculaveis
troca_tipo  = lista_local + lista_provas

# Transformar tipo de
enem_final[troca_tipo] = enem_final[troca_tipo].astype('string')

# Combinar os dicionários
dic_combinado = pd.concat([dic_imutaveis_l, dic_imutaveis_n], ignore_index=True)

enem_final[troca_tipo] = enem_final[troca_tipo].fillna('X')

AVISO: Q001 não tem código NULO no dicionário
AVISO: Q002 não tem código NULO no dicionário
AVISO: Q003 não tem código NULO no dicionário
AVISO: Q004 não tem código NULO no dicionário
AVISO: Q006 não tem código NULO no dicionário
AVISO: Q007 não tem código NULO no dicionário
AVISO: Q008 não tem código NULO no dicionário
AVISO: Q009 não tem código NULO no dicionário
AVISO: Q010 não tem código NULO no dicionário
AVISO: Q011 não tem código NULO no dicionário
AVISO: Q012 não tem código NULO no dicionário
AVISO: Q013 não tem código NULO no dicionário
AVISO: Q014 não tem código NULO no dicionário
AVISO: Q015 não tem código NULO no dicionário
AVISO: Q016 não tem código NULO no dicionário
AVISO: Q017 não tem código NULO no dicionário
AVISO: Q018 não tem código NULO no dicionário
AVISO: Q019 não tem código NULO no dicionário
AVISO: Q020 não tem código NULO no dicionário
AVISO: Q021 não tem código NULO no dicionário
AVISO: Q022 não tem código NULO no dicionário
AVISO: Q023 não tem código NULO no

In [ ]:
enem_final.to_parquet(output_path, index=False)

print(f"Realizado as transformações do arquivo {file} e salvo como {output_path}")

In [ ]:
def criar_dimensoes_do_dataframe(df_original, lista_variaveis=None):

    #Cria DataFrames separados (dimensões) para cada Nome_Variavel único.

    dataframes_criados = {}

    # Se não especificou lista, pega todos os valores únicos de Nome_Variavel
    if lista_variaveis is None:
        valores_unicos = df_original['Nome_Variavel'].unique()
    else:
        valores_unicos = lista_variaveis

    for valor in valores_unicos:
        # Filtra o DataFrame original onde Nome_Variavel == valor
        df_filtrado = df_original[df_original['Nome_Variavel'] == valor].copy()

        # Se encontrou dados para este valor, cria o DataFrame
        if not df_filtrado.empty:
            nome_dimensao = f"Dim_{valor}"
            dataframes_criados[nome_dimensao] = df_filtrado
            print(f"Dimensão '{nome_dimensao}' criada com {len(df_filtrado)} linhas")
        else:
            print(f"Aviso: Nenhuma linha encontrada para '{valor}'")

    return dataframes_criados


In [ ]:
def separar_dataframe_em_dimensoesv2(df_original, lista_variaveis=None, criar_variaveis_globais=True):
    #
    #Separa um DataFrame em vários DataFrames menores, criando variáveis separadas.

    #Parâmetros:
    #df_original: DataFrame com colunas 'Nome_Variavel', 'Categoria', 'Descricao_Categoria'
    #lista_variaveis: lista com nomes específicos para filtrar. Se None, usa todos os valores únicos
    #criar_variaveis_globais: Se True, cria variáveis globais (Dim_TP_SEXO, Dim_Q001, etc.)

    #Retorna:
    #dict: Dicionário com os DataFrames criados (independente de criar_variaveis_globais)

    import inspect

    # Obtém o frame do código que chamou esta função (para acessar suas variáveis globais)
    frame = inspect.currentframe().f_back
    globals_dict = frame.f_globals

    dataframes_criados = {}

    # Se não especificou lista, pega todos os valores únicos de Nome_Variavel
    if lista_variaveis is None:
        valores_unicos = df_original['Nome_Variavel'].unique()
    else:
        valores_unicos = lista_variaveis

    print(f"Separando DataFrame em {len(valores_unicos)} dimensões...")
    print("="*60)

    for valor in valores_unicos:
        # Filtra o DataFrame original onde Nome_Variavel == valor
        df_filtrado = df_original[df_original['Nome_Variavel'] == valor].copy().reset_index(drop=True)

        # Se encontrou dados para este valor, cria o DataFrame
        if not df_filtrado.empty:
            nome_dimensao = f"Dim_{valor}"
            dataframes_criados[nome_dimensao] = df_filtrado.reset_index(drop=True)

            # Se solicitado, cria a variável global
            if criar_variaveis_globais:
                globals_dict[nome_dimensao] = df_filtrado.reset_index(drop=True)
                print(f"✓ Variável '{nome_dimensao}' criada com {len(df_filtrado)} linhas")
            else:
                print(f"✓ Dimensão '{nome_dimensao}' criada com {len(df_filtrado)} linhas")
        else:
            print(f"⚠ Aviso: Nenhuma linha encontrada para '{valor}'")
        # Salvar como parquet se output_dir for especificado
        if output_dir:
                output_path = os.path.join(output_dir, f"{nome_dimensao}.parquet")
                df_filtrado.to_parquet(output_path, index=False)
                print(f"  Arquivo parquet salvo em: {output_path}")
    print("="*60)
    print(f"Processo concluído! {len(dataframes_criados)} DataFrames criados.")

    if criar_variaveis_globais:
        print("\nVariáveis criadas:")
        for nome in dataframes_criados.keys():
            print(f"  • {nome}")



    return dataframes_criados

In [ ]:
separar_dataframe_em_dimensoesv2(dic_combinado)

Separando DataFrame em 42 dimensões...
✓ Variável 'Dim_TP_SEXO' criada com 2 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/02-silver/Dim_TP_SEXO.parquet
✓ Variável 'Dim_Q001' criada com 8 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/02-silver/Dim_Q001.parquet
✓ Variável 'Dim_Q002' criada com 8 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/02-silver/Dim_Q002.parquet
✓ Variável 'Dim_Q003' criada com 6 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/02-silver/Dim_Q003.parquet
✓ Variável 'Dim_Q004' criada com 6 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/02-silver/Dim_Q004.parquet
✓ Variável 'Dim_Q006' criada com 17 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/02-silver/Dim_Q006.parquet
✓ Variável 'Dim_Q007' criada com 4 linhas
  Arquivo parquet salvo em: /content/drive/MyDrive/ENEM/Enem_Lakehouse/0

{'Dim_TP_SEXO':   Nome_Variavel Categoria Descricao_Categoria
 0       TP_SEXO         M           Masculino
 1       TP_SEXO         F           Feminino ,
 'Dim_Q001':   Nome_Variavel Categoria                                Descricao_Categoria
 0          Q001         A                                     Nunca estudou.
 1          Q001         B  Não completou a 4ª série/5º ano do Ensino Fund...
 2          Q001         C  Completou a 4ª série/5º ano, mas não completou...
 3          Q001         D  Completou a 8ª série/9º ano do Ensino Fundamen...
 4          Q001         E  Completou o Ensino Médio, mas não completou a ...
 5          Q001         F  Completou a Faculdade, mas não completou a Pós...
 6          Q001         G                         Completou a Pós-graduação.
 7          Q001         H                                           Não sei.,
 'Dim_Q002':   Nome_Variavel Categoria                                Descricao_Categoria
 0          Q002         A            

In [ ]:
Dim_TP_FAIXA_ETARIA

,Nome_Variavel,Categoria,Descricao_Categoria
0,TP_FAIXA_ETARIA,1,Menor de 17 anos
1,TP_FAIXA_ETARIA,2,17 anos
2,TP_FAIXA_ETARIA,3,18 anos
3,TP_FAIXA_ETARIA,4,19 anos
4,TP_FAIXA_ETARIA,5,20 anos
5,TP_FAIXA_ETARIA,6,21 anos
6,TP_FAIXA_ETARIA,7,22 anos
7,TP_FAIXA_ETARIA,8,23 anos
8,TP_FAIXA_ETARIA,9,24 anos
9,TP_FAIXA_ETARIA,10,25 anos


In [ ]:


# Get all variables in the current global scope
all_variables = dict(globals())

# Filter for variables that are pandas DataFrames
dataframe_variables = {name: var for name, var in all_variables.items() if isinstance(var, pd.DataFrame)}

# Print the names and types of the DataFrame variables
if dataframe_variables:
    print("DataFrames found in the current session:")
    for name, df in dataframe_variables.items():
        print(f"- {name} ({type(df)}) - Shape: {df.shape}")
else:
    print("No DataFrames found in the current session.")

DataFrames found in the current session:
- _ (<class 'pandas.core.frame.DataFrame'>) - Shape: (20, 3)
- __ (<class 'pandas.core.frame.DataFrame'>) - Shape: (2, 3)
- dic_imutaveis_l (<class 'pandas.core.frame.DataFrame'>) - Shape: (126, 3)
- dic_imutaveis_n (<class 'pandas.core.frame.DataFrame'>) - Shape: (97, 3)
- df_enem_base (<class 'pandas.core.frame.DataFrame'>) - Shape: (1500000, 76)
- df_enem (<class 'pandas.core.frame.DataFrame'>) - Shape: (1500000, 68)
- dic_combinado (<class 'pandas.core.frame.DataFrame'>) - Shape: (223, 3)
- dic_final (<class 'pandas.core.frame.DataFrame'>) - Shape: (265, 3)
- enem_final (<class 'pandas.core.frame.DataFrame'>) - Shape: (1500000, 68)
- _15 (<class 'pandas.core.frame.DataFrame'>) - Shape: (223, 3)
- Dim_TP_SEXO (<class 'pandas.core.frame.DataFrame'>) - Shape: (2, 3)
- Dim_Q001 (<class 'pandas.core.frame.DataFrame'>) - Shape: (8, 3)
- Dim_Q002 (<class 'pandas.core.frame.DataFrame'>) - Shape: (8, 3)
- Dim_Q003 (<class 'pandas.core.frame.DataFrame